In [1]:
# set up a GPU
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0, 1"


In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from datautils import GenDataset
from networkmodels import Network1, load_network, save_network

In [3]:
# test GPU working (output should be "Using -- cuda")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print( "Using --", device )

Using -- cuda


In [4]:
# load build data set 
filename = '../sim-data/population-sample.csv'

pred_vars_to_use = ["Age", "Sex", "MDD_hx", "Rx_hx", "Py_hx", "Sev", "Dur", "MDD_dx"]
target_var_to_use = ["Offer_Rx"]

# instantiate the data loader for the actual data
test_data = GenDataset( filename, pred_vars_to_use, target_var_to_use, trainflag = False )
test_loader = DataLoader(test_data, batch_size= len( test_data), shuffle = False, num_workers=2 )

# establish input/output dimensions
test_x, test_y = next(iter(test_loader))

input_size = test_x[0].shape[0]
output_size = test_y[0].shape[0]

print(f" Data Input size = {input_size}")
print(f" Data Target size = {output_size}")

 Data Input size = 8
 Data Target size = 1


In [5]:
# instantiate network
model = Network1( input_size, output_size)

# restore state from epoch 29
load_network( model, 'net_29.pth')
print(f"Model Activations Total = {model.total_activations}")
# print( model.state_dict()["linear.weight"].numpy() )
# print( model.state_dict()["linear.bias"].numpy() )

Model Activations Total = 2


In [20]:
dataset = test_loader.dataset
this_input = dataset[1][0]
this_label = dataset[1][1]

acts = np.zeros( shape = ( len(dataset), model.total_activations ) )
sample_data = np.zeros( shape = ( len(dataset), len( this_input ) ) )
sample_target = np.zeros( shape = ( len(dataset), 1 ) )

for i in range(len( dataset ) ):
    this_input = dataset[i][0]
    this_label = dataset[i][1]
    activations_vector = model.get_activations( this_input )
    acts[i,:] = activations_vector
    sample_data[i,:] = this_input
    sample_target[i,:] = this_label 

df = pd.DataFrame( np.hstack([sample_data, acts, sample_target]) )
df.to_csv('../out-data/acts-one-layer-model.csv', header = False)


In [17]:
print(  )

[[21.          0.          1.         ...  2.40099692  0.91690332
   1.        ]
 [37.          1.          0.         ...  1.84118295  0.86308855
   0.        ]
 [46.          0.          0.         ...  4.89149904  0.99254578
   1.        ]
 ...
 [77.          1.          0.         ...  1.42974591  0.8068617
   1.        ]
 [49.          1.          0.         ...  2.24327135  0.90406853
   1.        ]
 [35.          0.          1.         ... -1.1418252   0.24198541
   0.        ]]


In [22]:



# print( np.vstack([c,c]) )

# a = np.array([1, 2, 3])
# b = np.array([5, 6])
# a = np.concatenate([a, b])

# c = np.array([3.4,1.2,8.9])
# a = np.concatenate([a,c])

print(type(this_input))




tensor([2.5366])
tensor([0.9267])
<class 'torch.Tensor'>
